In [1]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from scipy.io import loadmat
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import time

In [2]:
start=time.time()
dat = loadmat('./高光谱数据集/KSC.mat')['KSC']
lab = loadmat('./高光谱数据集/KSC_gt.mat')['KSC_gt']

dat = dat.reshape(-1, 176)  
lab = lab.reshape(-1)
print(dat.shape)

data,label=[],[]
for i in range(dat.shape[0]):
    if lab[i].all()!=0:
        data.append(dat[i,:])
        label.append(lab[i])
        
data=np.array(data)
label=np.array(label)
        


(314368, 176)


In [3]:
data=preprocessing.StandardScaler().fit_transform(data)
print ('shape={}'.format(data.shape))

shape=(5211, 176)


In [4]:
selector = VarianceThreshold()  # 实例化，不填参数默认方差为0
data = selector.fit_transform(data)
print(data.shape)

(5211, 176)


In [5]:
median_num = np.median(data)
data = VarianceThreshold(median_num).fit_transform(data)
print(data.shape)

(5211, 176)


In [6]:
acc = cross_val_score(KNN(), data, label, cv=5).mean()   
print("accuracy:{},time:{}".format(acc,time.time()-start))

accuracy:0.862032598274209,time:2.707597494125366


In [7]:
def select_k(data,label):
    results = SelectKBest(f_classif,k=120).fit(data,label)
    print(results)
    features = pd.DataFrame({
    "score": results.scores_,
    "pvalue": results.pvalues_,
    "select": results.get_support()
    })
    features.sort_values("score", ascending=False)
    print(features)
    index=results.get_support(indices=True)
    print(index)
    return index

In [8]:
def rfe(data,label):
    results=RFE(estimator=LogisticRegression(),n_features_to_select=10)
    print(results)
    results.fit(data,label)

In [9]:
index= select_k(data,label)
#index=rfe(data,label)

SelectKBest(k=120)
          score        pvalue  select
0      5.342886  4.571589e-09   False
1    628.181103  0.000000e+00   False
2    564.241504  0.000000e+00   False
3    577.317465  0.000000e+00   False
4    632.796860  0.000000e+00   False
..          ...           ...     ...
171    2.236971  8.255060e-03   False
172    0.911922  5.338891e-01   False
173    6.680017  4.866397e-12   False
174    3.143885  1.767045e-04   False
175    6.598877  7.413556e-12   False

[176 rows x 3 columns]
[ 32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49
  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 139 140 142 143 144 145 146 147 148
 149 150 151 152 153 154 157 158 161 162 164 165]


In [10]:
def select_index_data(index,data,label):
    data_after=[]
    for i in index:
        data_after.append(data[:,i])
    data_after=np.array(data_after).transpose()
    print(data_after.shape)
    print(label.shape)
    return train_test_split(data_after,label,test_size=0.3, random_state=1)

In [11]:
train_data,test_data,train_label,test_label=select_index_data(index,data,label)

(5211, 120)
(5211,)


In [12]:
def measure_feature(train_data,test_data,train_label,test_label,gamma,c):
    clf = sklearn.svm.SVC(kernel='poly',gamma=gamma,C=c)
    clf.fit(train_data,train_label)
    predict = clf.predict(test_data)
    clf.get_params(deep=True)
    acc= sklearn.metrics.accuracy_score(test_label,predict)
    f1=sklearn.metrics.f1_score(test_label,predict,average='micro')
    recall=metrics.recall_score(test_label,predict,average='micro')
    precision=metrics.precision_score(test_label,predict,average='micro')
    return acc,f1,recall,precision

In [13]:
gamma,c=0.125,60
train_acc,train_f1,train_recall,train_precision=measure_feature(train_data,train_data,train_label,train_label,gamma,c)
test_acc,test_f1,test_recall,test_precision=measure_feature(train_data,test_data,train_label,test_label,gamma,c)

print(train_acc,test_acc)
print(train_f1,test_f1)
print(train_recall,test_recall)
print(train_precision,test_precision)

0.9958870304359748 0.9213554987212276
0.9958870304359748 0.9213554987212276
0.9958870304359748 0.9213554987212276
0.9958870304359748 0.9213554987212276


In [14]:
print('训练集准确率为{:.4f}，测试集准确率为{:.4f}'.format(train_acc,test_acc))

训练集准确率为0.9959，测试集准确率为0.9214
